In [1]:
# Installation Step (if needed)
!python3 -m pip install mkl mkl-fft
!python3 -m pip install "snowflake-snowpark-python[pandas]"
!python3 -m pip install "snowflake[ml]"
print("Packages Installed")

Packages Installed


In [2]:
#authenticate and create Snowpark Session
import json
from getKeyPair import get_private_key
from snowflake.snowpark import Session

with open('creds.json') as f:
    sf_params=json.load(f)
    sf_params['private_key']=get_private_key(sf_params['private_key_file'])
session = Session.builder.configs(sf_params).create()
print(session)

<snowflake.snowpark.session.Session: account="sfsenorthamerica-pfg_lab", role="PRINCIPAL", database="QUICKSTART_SF0001_BASE", schema="PUBLIC", warehouse="LAB">


In [48]:
#Specify Parameters to define Export
TABLE='SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER'
DATASET_STAGE="@QUICKSTART_SF0001_BASE.PUBLIC.S3_EAF"
DATASET_STAGE_PATH=DATASET_STAGE+"/eaf/"
TARGET_FILE_SIZE = 32 * 2**20 # ~32MB
DATASET_NAME="mydataset_1"
FILE_FORMAT='parquet'
SINGLE_FILE=False

In [22]:
# Export Table to External Stage (S3 Bucket)
df1 = session.table(TABLE)
s3_files=df1.write.copy_into_location(
                location=f"'{DATASET_STAGE_PATH}'",
                file_format_type=f"'{FILE_FORMAT}'",
                header=True,
                partition_by=f"'{DATASET_NAME}'",
                max_file_size=TARGET_FILE_SIZE,
                single=SINGLE_FILE,
                detailed_output=True)
for row in s3_files:
    print(row.as_dict())

{'PARTITION_NAME': 'mydataset_1', 'FILE_NAME': 'mydataset_1/data_01b59868-0205-881b-0049-5607009c4656_006_4_0.snappy.parquet', 'FILE_SIZE': 13295113, 'ROW_COUNT': 150000}


In [32]:
f=session.sql(f"list {DATASET_STAGE_PATH}").collect()
for row in f:
    print(row.as_dict())

{'name': 's3://smaser-east-snowflake/eaf/mydataset_1/data_01b5985a-0205-880c-0049-5607009c75f6_006_4_0.snappy.parquet', 'size': 13295113, 'md5': 'b11c9f5cc8fc29d04cc2ae26802dfd68', 'last_modified': 'Thu, 11 Jul 2024 10:34:17 GMT'}
{'name': 's3://smaser-east-snowflake/eaf/mydataset_1/data_01b5985f-0205-8816-0049-5607009c0612_006_4_0.snappy.parquet', 'size': 13295113, 'md5': 'b11c9f5cc8fc29d04cc2ae26802dfd68', 'last_modified': 'Thu, 11 Jul 2024 10:40:01 GMT'}
{'name': 's3://smaser-east-snowflake/eaf/mydataset_1/data_01b59860-0205-8813-0049-5607009c3e42_006_4_0.snappy.parquet', 'size': 13295113, 'md5': 'b11c9f5cc8fc29d04cc2ae26802dfd68', 'last_modified': 'Thu, 11 Jul 2024 10:40:09 GMT'}
{'name': 's3://smaser-east-snowflake/eaf/mydataset_1/data_01b59861-0205-8813-0049-5607009c3e46_006_4_0.snappy.parquet', 'size': 13295113, 'md5': 'b11c9f5cc8fc29d04cc2ae26802dfd68', 'last_modified': 'Thu, 11 Jul 2024 10:41:11 GMT'}
{'name': 's3://smaser-east-snowflake/eaf/mydataset_1/data_01b59861-0205-8815

# Other Variations & Common Commands

#To test larger dataset: <br>
df1 = session.table('SNOWFLAKE_SAMPLE_DATA.TPCH_SF1000.CUSTOMER')

#To randomly distribute data: <br>
import snowflake.snowpark.functions as F
df1 = session.table(TABLE).order_by(F.random())

#Sample Table:<br>
session.table(TABLE).sample(n=10).show()

#Subset of Columns:<br>
session.table(TABLE).select(["C_CUSTKEY", "C_NAME"]).limit(10).show()

#Filter / Where clause:<br>
import snowflake.snowpark.functions as F
session.table(TABLE).filter(F.col("C_CUSTKEY") <= 60010).show()

#using SQL:<br>
df1 = session.sql(f"SELECT * FROM {TABLE}").collect()

#using SQL to perform COPY INTO entirely:<br>
session.sql(f"COPY INTO {DATASET_STAGE_PATH} from {TABLE} PARTITION BY ('{DATASET_NAME}') FILE_FORMAT=(TYPE=parquet) HEADER=TRUE MAX_FILE_SIZE ={TARGET_FILE_SIZE} SINGLE={SINGLE_FILE} INCLUDE_QUERY_ID=true").show()


https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/1.18.0/snowpark/api/snowflake.snowpark.DataFrameWriter.copy_into_location
